In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results = 2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:
    def __init__(self, model, tools, system = ""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("LLM", self.call_openai)
        graph.add_node("Action", self.take_action)
        graph.add_conditional_edges(
            "LLM", 
            self.exists_action,
            {True: "Action", False: END}
        )
        graph.add_edge("Action", "LLM") 
        graph.set_entry_point("LLM") 
        self.graph = graph.compile() 
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages'] 
        if self.system:   
            messages = [SystemMessage(content = self.system)] + messages  #prepend to list of messages
        message = self.model.invoke(messages)  
        return {'messages': [message]} 

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print("\n...Bad tool name...")
                result = "Bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id = t['id'], name = t['name'], content = str(result)))
        print("back to the model!")
        return {'messages': results}

    #define conditonal edge
    def exists_action(self, state: AgentState):
        #take the last message
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

In [5]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  
abot = Agent(model, [tool], system=prompt)  

In [6]:
messages = [HumanMessage(content = "What is the weather in sf?")] 
result = abot.graph.invoke({"messages": messages}) 

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_0KREg4Y6KNgtUOElRHCjHTO6', 'type': 'tool_call'}
back to the model!


In [7]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0KREg4Y6KNgtUOElRHCjHTO6', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a2f5d45a-caa8-45ca-847a-482328fb5da3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_0KREg4Y6KNgtUOElRHCjHTO6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 21, 'total_tokens': 174}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'l

In [8]:
result['messages'][-1].content

'The current weather in San Francisco is sunny with a temperature of 56.3°F (13.5°C). The wind speed is 10.4 km/h coming from the southwest direction. The humidity is at 84%, and the visibility is 6.0 miles.'

**Two Part Question**

In [10]:
messages = [HumanMessage(content = "What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_4XX1WnYbsxARGO73GODjtuWG', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_lbqVwSmLbO6vl3xLDUKt25bK', 'type': 'tool_call'}
back to the model!


In [11]:
result['messages'][-1].content

'The current weather in San Francisco is sunny with a temperature of 56.3°F (13.5°C). The wind speed is 10.4 kph coming from the southwest direction. The humidity level is at 84%.\n\nIn Los Angeles, the weather is also sunny with a temperature of 75.3°F (24.0°C). The wind speed is 5.4 kph coming from the south-southeast direction. The humidity level is at 57%.'

In [12]:
query = "Who won the super bowl in 2024? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl winner 2024'}, 'id': 'call_1vHMe5OYlJCxSYWJh73jb8f6', 'type': 'tool_call'}
back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2024'}, 'id': 'call_wvKgp3zZzwaDkyiWyNwU8xSV', 'type': 'tool_call'}
back to the model!


In [13]:
print(result['messages'][-1].content)

### Super Bowl Winner 2024
The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers in an overtime victory.

### GDP of Missouri
The real gross domestic product (GDP) of Missouri in 2024 is not directly available, but in 2023, it was approximately $344.12 billion USD. Given the context, it is likely to have seen a modest increase in 2024.
